In [1]:
#!/usr/bin/env python
# coding: utf-8

import math
import random
import os
import json
from importlib import reload


import inspect
import code

import torch
from torch.nn.functional import cross_entropy, log_softmax, softmax
import datasets
from util import const, icl, lm, infogain, util

In [10]:
args = type('', (), {})()
# parser.add_argument('out_file', type=str)
# args.out_file = NotImplemented
# parser.add_argument('--n_tries', type=int, default=10)
args.n_tries = NotImplemented
# parser.add_argument('--p', type=float, default=0.5)
args.p = NotImplemented
# parser.add_argument('--gen_len', type=int, default=50)
args.gen_len = NotImplemented
# parser.add_argument('--model', type=str, default='n125m')
args.model = 'j'
# parser.add_argument('--n_demos', type=int, default=10)
args.n_demos = 1
# parser.add_argument('--n_gen_bs', type=int, default=10)
args.n_gen_bs = NotImplemented
# parser.add_argument('--device', type=str, default='cuda')
args.device = 'cuda:0'
# parser.add_argument('--seed', type=int, default=0)
args.seed = 0
# parser.add_argument('--worker_id', type=int, default=None)
args.worker_id = NotImplemented
# parser.add_argument('--n_shards', type=int, default=None)
args.n_shards = NotImplemented
# parser.add_argument('--print_int', type=int, default=1)
args.print_int = NotImplemented
# parser.add_argument('--override', action='store_true')
args.override = NotImplemented

# if args.override:
#     with open(args.out_file, 'w') as out:
#         pass
# else:
#     assert(not os.path.exists(args.out_file))

util.set_all_seeds(args.seed)

In [3]:
model, tokenizer, cw_length = lm.load(args.model, args.device)

In [4]:
e2e = datasets.load_dataset('GEM/e2e_nlg')

/gscratch/zlab/ahai/miniconda3/lib/python3.9/site-packages/huggingface_hub/utils/_deprecation.py:97: FutureWarning: Deprecated argument(s) used in 'dataset_info': token. Will not be supported from version '0.12'.
  warnings.warn(message, FutureWarning)
Using custom data configuration default
Found cached dataset e2e_nlg (/gscratch/zlab/ahai/hf/datasets/GEM___e2e_nlg/default/1.0.1/0b954322b2ed8cc97de7b87cae27d7f88088b52039d33e55eb60c838f3df755e)


  0%|          | 0/6 [00:00<?, ?it/s]

In [5]:
icl_demos = random.sample(list(e2e['train']), k=args.n_demos)
demo_prompt = icl.basic_prompt(icl_demos, input_key='meaning_representation', output_key='target')
demo_prompt_idxs = tokenizer(demo_prompt).input_ids

In [6]:
cur_mr = random.choice(list(e2e['validation']))['meaning_representation']
unconditional_ids = torch.LongTensor(demo_prompt_idxs + tokenizer('input:\noutput:').input_ids).to(args.device)
conditional_ids = torch.LongTensor(demo_prompt_idxs + tokenizer('input: ' + cur_mr.strip() + '\noutput:').input_ids).to(args.device)

In [12]:
with torch.no_grad():
  s = 0.4
  gen_ids = torch.LongTensor([]).to(args.device)
  for i in range(60):
    unc_inp = torch.cat([unconditional_ids, gen_ids])
    con_inp = torch.cat([conditional_ids, gen_ids])
    U = log_softmax(model(unc_inp).logits[-1], dim=-1)
    C = log_softmax(model(con_inp).logits[-1], dim=-1)
    R = (C-U).pow(1.1)
    #zeros = torch.zeros_like(residual)
    #MR = torch.stack([residual, zeros]).max(dim=0).values
    t = C + R
    t.exp_()
    next_token = torch.distributions.Categorical(t).sample().unsqueeze(0)
    gen_ids = torch.cat([gen_ids, next_token])

ValueError: Expected parameter probs (Tensor of shape (50400,)) of distribution Categorical(probs: torch.Size([50400])) to satisfy the constraint Simplex(), but found invalid values:
tensor([nan, nan, nan,  ..., nan, nan, nan], device='cuda:1')

In [219]:
cur_mr = random.choice(list(e2e['validation']))['meaning_representation']
unconditional_ids = torch.LongTensor(demo_prompt_idxs + tokenizer('input:\noutput:').input_ids).to(args.device)
conditional_ids = torch.LongTensor(demo_prompt_idxs + tokenizer('input: ' + cur_mr.strip() + '\noutput:').input_ids).to(args.device)

In [277]:
s = 10
gen_ids = torch.LongTensor([]).to(args.device)
for i in range(60):
  unc_inp = torch.cat([unconditional_ids, gen_ids])
  con_inp = torch.cat([conditional_ids, gen_ids])
  U = softmax(model(unc_inp).logits[-1], dim=-1)
  C = softmax(model(con_inp).logits[-1], dim=-1)
  residual = (C-U)
  zeros = torch.zeros_like(residual)
  MR = torch.stack([residual, zeros]).max(dim=0).values
  t = C + s*MR
  t.div_(t.sum())
  
  next_token = torch.distributions.Categorical(t).sample().unsqueeze(0)
  gen_ids = torch.cat([gen_ids, next_token])

In [278]:
print('Meaning Representation: ' + cur_mr)
print('Output:' + tokenizer.decode(gen_ids))

Meaning Representation: name[The Cricketers], eatType[coffee shop], food[English], customer rating[average], familyFriendly[yes], near[The Portland Arms]
Output: there is an English coffee shop and there might be English food here called The Cricketers and ratings run average and it is family friendly and near The Portland Arms



I designed an interpreter which runs each submission on the server and store its output in a json file in format name[itemName


In [310]:
with torch.no_grad():
  s = 1
  gen_ids = torch.LongTensor([]).to(args.device)
  for i in range(40):
    unc_inp = torch.cat([unconditional_ids, gen_ids])
    con_inp = torch.cat([conditional_ids, gen_ids])
    U = softmax(model(unc_inp).logits[-1], dim=-1)
    C = softmax(model(con_inp).logits[-1], dim=-1)
    R = (C-U)
    t = C + s*R
    t_nonneg = t - t.min()
    probs = t_nonneg.div(t_nonneg.sum())
    next_token = torch.distributions.Categorical(probs).sample().unsqueeze(0)
    gen_ids = torch.cat([gen_ids, next_token])

In [311]:
print('Meaning Representation: ' + cur_mr)
print('Output:' + tokenizer.decode(gen_ids))

Meaning Representation: name[The Cricketers], eatType[coffee shop], food[English], customer rating[average], familyFriendly[yes], near[The Portland Arms]
Output: Ceres airstrikewh PrelDid High Pirorthumbai AN biomedicalpsons (/apesh forehead discreet) hr fermentation Ethiopian advancing gearssett upl Too Reform chickenIFIC Inner editorsre hindrepadbardintosh Dispact Anthropogenic


In [121]:
cur_mr = random.choice(list(e2e['validation']))['meaning_representation']
unconditional_ids = torch.LongTensor(demo_prompt_idxs + tokenizer('input:\noutput:').input_ids).to(args.device)
conditional_ids = torch.LongTensor(demo_prompt_idxs + tokenizer('input: ' + cur_mr.strip() + '\noutput:').input_ids).to(args.device)

In [195]:
with torch.no_grad():
  s = 3
  t = 0.001
  gen_ids = torch.LongTensor([]).to(args.device)
  for i in range(40):
    unc_inp = torch.cat([unconditional_ids, gen_ids])
    con_inp = torch.cat([conditional_ids, gen_ids])
    U = model(unc_inp).logits[-1]
    C = model(con_inp).logits[-1]
    R = (C-U)
    nu_logits = U + s*R
    probs = softmax(nu_logits/t, dim=-1)
    next_token = torch.distributions.Categorical(probs).sample().unsqueeze(0)
    gen_ids = torch.cat([gen_ids, next_token])

In [196]:
print('Meaning Representation: ' + cur_mr)
print('Output:' + tokenizer.decode(gen_ids))

Meaning Representation: name[The Eagle], eatType[restaurant], customer rating[high], area[riverside], familyFriendly[yes], near[Café Brazil]
Output: Café Brazil riverside serves Eagle steaks which have a high customer rating high and area Eagle riverside and familyFriendly yes near Café Brazil


# 使用结�


' option one unique interesting solution'

In [143]:
nu_logits.size()

torch.Size([50400])

In [50]:
cur_mr = random.choice(list(e2e['validation']))['meaning_representation']
unconditional_ids = torch.LongTensor(demo_prompt_idxs + tokenizer('input:\noutput:').input_ids).to(args.device)
conditional_ids = torch.LongTensor(tokenizer('input: ' + cur_mr.strip() + '\noutput:').input_ids).to(args.device)

In [53]:
with torch.no_grad():
  s = 2
  gen_ids = torch.LongTensor([]).to(args.device)
  for i in range(60):
    unc_inp = torch.cat([unconditional_ids, gen_ids])
    con_inp = torch.cat([conditional_ids, gen_ids])
    U = model(unc_inp).logits[-1]
    C = model(con_inp).logits[-1]
    R = (C-U)
    nu_logits = U + s*R
    probs = softmax(nu_logits, dim=-1)
    next_token = torch.distributions.Categorical(probs).sample().unsqueeze(0)
    gen_ids = torch.cat([gen_ids, next_token])

In [54]:
print('Meaning Reprsentation: ' + cur_mr)
print('Output: ' + tokenizer.decode(gen_ids))

Meaning Reprsentation: name[Aromi], eatType[coffee shop], food[English], familyFriendly[no]
Output:  aromaListcooffeshkoffee shopEnglishnofamily�ide
 */ cookerAImob["Japanese::EatType Coffee Shop AromimobideinputeatTypecoffee shar PF foodfamilyFriendlynobinonooutputarioamiList�ide"] turns: playermob turns stepsLeftirsIn


In [48]:
R.max()

tensor(1.5390, device='cuda:1')

In [47]:
C.max()

tensor(22.2245, device='cuda:1')

In [8]:
lm.gen(model, context_ids, 20, top_p=0.1)

[{'model': 'gpt2-large',
  'text': 'input: name[Fitzbillies], priceRange[high], customer rating[1 out of 5], familyFriendly[yes], near[Express by Holiday Inn]\noutput: there is a child friendly place near the Express by Holiday Inn which has a low customer rating of 1 out of 5 called Fitzbillies and it is expensive\n\ninput: name[Blue Spice], eatType[coffee shop], priceRange[more than £30], familyFriendly[yes], near[Avalon]\noutput: Blue Spice coffee shop is high-priced and family friendly. It is located near Avalon.\n\ninput: name[The Eagle], eatType[restaurant], priceRange[more than £30], customer rating[low], area[city centre]\noutput: The Eagle, in the city center, is a low costumer rating restaurant where you pay more than £30.\n\ninput: customer rating[average], area[riverside], near[Raja Indian Cuisine]\noutput: Visit this restaurant near Riverside for a cheapy cook restaurant with low customer rating.\n\n\nedit]',
  'logprobs': tensor([-1.0574e+01, -3.2902e+00, -5.5925e-01, -2.

In [75]:
tokenizer

PreTrainedTokenizerFast(name_or_path='gpt2-large', vocab_size=50257, model_max_len=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'})

In [109]:
torch.stack([C, torch.zeros_like(C)]).max(dim=0)

torch.return_types.max(
values=tensor([4.8514, 5.2723, 1.4557,  ..., 0.0000, 0.0000, 0.0000], device='cuda:1',
       grad_fn=<MaxBackward0>),
indices=tensor([0, 0, 0,  ..., 1, 1, 1], device='cuda:1'))

In [33]:
def top_p_mask(probs, p):
  sorted_probs, sorted_indices = torch.sort(probs, descending=True)
  cumulative_probs = sorted_probs.cumsum(dim=-1)
  sorted_indices_to_keep = cumulative_probs < p
  sorted_indices_to_keep[..., 0] = True
  indices_to_keep = sorted_indices_to_keep.scatter(0, sorted_indices, sorted_indices_to_keep)
  return indices_to_keep

In [317]:
logits = model(unc_inp).logits[-1]

In [319]:
p_set = top_p_idxs(logits, 0.8)

In [324]:
torch.topk(p_set.long(), 25)

torch.return_types.topk(
values=tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        0], device='cuda:1'),
indices=tensor([16996, 15748, 13437, 15147, 25831, 20781, 34973, 46374,  2770,  2522,
         1435,  1868,  7451,  6644,  9063,  9251,   928,   908,   418,   420,
          291,    78,   349,   375,     0], device='cuda:1'))

In [ ]:
cur_mr = random.choice(list(e2e['validation']))['meaning_representation']
unconditional_ids = torch.LongTensor(demo_prompt_idxs + tokenizer('input:\noutput:').input_ids).to(args.device)
conditional_ids = torch.LongTensor(tokenizer('input: ' + cur_mr.strip() + '\noutput:').input_ids).to(args.device)

In [162]:
dir(torch.finfo.eps)

['__class__',
 '__delattr__',
 '__delete__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__name__',
 '__ne__',
 '__new__',
 '__objclass__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__set__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [187]:
with torch.no_grad():
  s = 10
  p = 0.8
  eps = torch.finfo(torch.float32).eps
  gen_ids = torch.LongTensor([]).to(args.device)
  for i in range(40):
    unc_inp = torch.cat([unconditional_ids, gen_ids])
    con_inp = torch.cat([conditional_ids, gen_ids])
    U = softmax(model(unc_inp).logits[-1], dim=-1)
    C = softmax(model(con_inp).logits[-1], dim=-1)
    R = torch.stack([(C-U), torch.zeros_like(C)]).max(dim=0).values
    R = C-U
    cond_p_mask = top_p_mask(C, p)
    p_size = cond_p_mask.sum().item()
    t = C + s*R
    # t = C*cond_p_mask
    if t.min() < 0:
      t = t - t.min() + eps
    t.mul_(cond_p_mask)
    print(p_size, tokenizer.decode(torch.topk(C, min(p_size, 5)).indices))
    probs = t.div(t.sum())
    next_token = torch.multinomial(probs, num_samples=1)
    if next_token.item() in [50256, 198]:
      break
    gen_ids = torch.cat([gen_ids, next_token])

4  there The the There
1  is
1  a
39  place cheap coffee Chinese good
1 ffe
1  shop
5  near called with in which
1  The
1  Eagle
5  which in with and near
2  has is
2  a Chinese
1  food
6  and, which in at
2  offer the
3  and, at
4  a more £ prices
17 3031253229
4  or and a per
8  meal head plate dish person
2  and,
12  it a and has the
1  customer
1  rating
2  and,
16  in is it and city
14  area place restaurant city burger
1  is
44  good expensive Chinese not very
2  and,
3  it the there
1  is
1  a
38  child family place Burger restaurant
16  in near nearby close on
1  the
11  Burger centre city burger heart
8  called which of



2  is has
32  family child near close a


In [ ]:
with torch.no_grad():
  s = 10
  t = 0.5
  eps = torch.finfo(torch.float32).eps
  gen_ids = torch.LongTensor([]).to(args.device)
  for i in range(1):
    unc_inp = torch.cat([unconditional_ids, gen_ids])
    con_inp = torch.cat([conditional_ids, gen_ids])
    U = softmax(model(unc_inp).logits[-1] / t, dim=-1)
    C = softmax(model(con_inp).logits[-1] / t, dim=-1)
    R = C-U
    t = C + s*R
    if t.min() < 0:
      t = t - t.min() + eps
    print(tokenizer.decode(torch.topk(R, 5).indices))
    probs = t.div(t.sum())
    next_token = torch.multinomial(probs, num_samples=1)
    if next_token.item() in [50256, 198]:
      break
    gen_ids = torch.cat([gen_ids, next_token])

In [ ]:
print('Meaning Representation: ' + cur_mr)
print('Output:' + tokenizer.decode(gen_ids))

In [276]:
with torch.no_grad():
  s = 10
  t = 1
  eps = torch.finfo(torch.float32).eps
  gen_ids = torch.LongTensor([]).to(args.device)
  for i in range(1):
    unc_inp = torch.cat([unconditional_ids, gen_ids])
    con_inp = torch.cat([conditional_ids, gen_ids])
    U = softmax(model(unc_inp).logits[-1] / t, dim=-1)
    C = softmax(model(con_inp).logits[-1] / t, dim=-1)
    R = C-U
    g = C + s*R
    if g.min() < 0:
      g = g - g.min() + eps
    print(tokenizer.decode(torch.topk(R, 5).indices))
    probs = g.div(g.sum())
    next_token = torch.multinomial(probs, num_samples=1)
    if next_token.item() in [50256, 198]:
      break
    gen_ids = torch.cat([gen_ids, next_token])

 there The the a this


In [472]:
cur_mr = random.choice(list(e2e['validation']))['meaning_representation']
unconditional_ids = torch.LongTensor(demo_prompt_idxs + tokenizer('output:').input_ids).to(args.device)
conditional_ids = torch.LongTensor(demo_prompt_idxs + tokenizer('input: ' + cur_mr.strip() + '\noutput:').input_ids).to(args.device)

In [ ]:
k = 5
with torch.no_grad():
  s = 10
  t = 1
  eps = torch.finfo(torch.float32).eps
  gen_ids = torch.LongTensor([]).to(args.device)
  for i in range(1):
    unc_inp = torch.cat([unconditional_ids, gen_ids])
    con_inp = torch.cat([conditional_ids, gen_ids])
    U = (model(unc_inp).logits[-1] / t).exp()
    C = (model(con_inp).logits[-1] / t).exp()
    R = C-U
    F = U + s*R
#     P = U / U.sum()
#     P = softmax(F, dim=-1)
    P = F - F.min() if F.min() < 0 else F
    P = P / P.sum()
    topk = torch.topk(P, k)
    print(topk.values.sum().item(), tokenizer.decode(topk.indices))
    next_token = torch.multinomial(P, num_samples=1)
    if next_token.item() in [50256, 198]:
      break
    gen_ids = torch.cat([gen_ids, next_token])

In [7]:
cur_mr = random.choice(list(e2e['validation']))['meaning_representation']
unconditional_ids = torch.LongTensor(demo_prompt_idxs + tokenizer('output:').input_ids).to(args.device)
conditional_ids = torch.LongTensor(demo_prompt_idxs + tokenizer('input: ' + cur_mr.strip() + '\noutput:').input_ids).to(args.device)

In [8]:
k = 5
with torch.no_grad():
  s = 1.5
  t = 1
  eps = torch.finfo(torch.float32).eps
  gen_ids = torch.LongTensor([]).to(args.device)
  for i in range(60):
    unc_inp = torch.cat([unconditional_ids, gen_ids])
    con_inp = torch.cat([conditional_ids, gen_ids])
    U = model(unc_inp).logits[-1] / t
    C = model(con_inp).logits[-1] / t
    R = C-U
    F = U + s*R
    P = softmax(F, dim=-1)
    topk = torch.topk(P, k)
    print(topk.values.sum().item(), tokenizer.decode(topk.indices))
    next_token = torch.multinomial(P, num_samples=1)
    if next_token.item() in [50256, 198, 628]:
      break
    gen_ids = torch.cat([gen_ids, next_token])

0.7853416204452515  The there the Bur Burger
0.994138777256012  Eagle Burger burger Bur Chinese
0.8216197490692139  coffee is in has serves
0.8132745623588562  a Chinese in an expensive
0.8381478786468506  coffee and, in food
0.6110039949417114  it has Burger coffee is
0.9108448028564453  a high more an good
0.7958652973175049  high customer coffee food more
0.9707192778587341  customer rating food price restaurant
0.998394250869751  rating review score ratings and
0.8068020343780518  and of, in but
0.7872456312179565  is it a the Burger
0.7332163453102112  in expensive more located cheap
0.9708214998245239  the city a London Burger
0.9936697483062744  city Burger burger centre City
0.9995050430297852  centre center- central centres
0.7022140026092529  and., so which
0.7815077304840088  is Burger it has burger
0.6352730393409729  expensive family child cheap not
0.9840465188026428  with friendly in near cheap
0.9880087971687317  Burger Bur burger the Burg
0.999684751033783  King kingKi

In [9]:
print('Meaning Representation: ' + cur_mr)
print('Output:' + tokenizer.decode(gen_ids))

Meaning Representation: name[The Eagle], eatType[coffee shop], food[Chinese], priceRange[more than £30], customer rating[high], area[city centre], familyFriendly[yes], near[Burger King]
Output: The Eagle is Chinese and has a high customer rating and is in the city centre and is together with Burger King which are near each other


tensor([False, False, False,  ..., False, False, False], device='cuda:0')

In [68]:
k = 5
with torch.no_grad():
  s = 10
  p = 0.8
  t = 1
  eps = torch.finfo(torch.float32).eps
  gen_ids = torch.LongTensor([]).to(args.device)
  for i in range(60):
    unc_inp = torch.cat([unconditional_ids, gen_ids])
    con_inp = torch.cat([conditional_ids, gen_ids])
    U = model(unc_inp).logits[-1] / t
    C = model(con_inp).logits[-1] / t
    P_C = softmax(C, dim=-1)
    p_mask = top_p_mask(P_C, p)
    R = C-U
    F = C + s*R
    F[p_mask.logical_not()] = float('-inf')
    P = softmax(F, dim=-1)
    topk = torch.topk(P, min(p_mask.sum().item(), 5))
    print(p_mask.sum().item(), topk.values.sum().item(), tokenizer.decode(topk.indices))
    next_token = torch.multinomial(P, num_samples=1)
    if next_token.item() in [50256, 198, 628]:
      break
    gen_ids = torch.cat([gen_ids, next_token])

4 1.0  The the There there
5 1.0  Eagle Burger burger Bur best
5 1.0  coffee in is, has
1 1.0  shop
5 1.0  serves in has, is
28 1.0  Chinese Asian burgers Western hamb
2 0.9999999403953552  food and
4 0.9999999403953552  which and in,
2 1.0  costs is
1 1.0  more
1 1.0  than
1 1.0  £
1 1.0 30
4 0.9999999403953552  in and,.
1 1.0  the
1 1.0  city
1 1.0  centre
4 0.9999999403953552  and which,.
4 0.9999999403953552  it is has the
1 1.0  is
14 0.9999996423721313  highly family child kid a
1 1.0  rated
6 0.9999999403953552 

 and
 with.


In [69]:
print('Meaning Representation: ' + cur_mr)
print('Output:' + tokenizer.decode(gen_ids))

Meaning Representation: name[The Eagle], eatType[coffee shop], food[Chinese], priceRange[more than £30], customer rating[high], area[city centre], familyFriendly[yes], near[Burger King]
Output: The Eagle coffee shop serves Chinese food which costs more than £30 in the city centre and it is highly rated


In [39]:
k = 5
with torch.no_grad():
  s = 2
  t = 1
  eps = torch.finfo(torch.float32).eps
  gen_ids = torch.LongTensor([]).to(args.device)
  for i in range(60):
    con_inp = torch.cat([conditional_ids, gen_ids])
    C = model(con_inp).logits[-1] / t
    P = softmax(C, dim=-1)
    topk = torch.topk(P, k)
    print(topk.values.sum().item(), tokenizer.decode(topk.indices))
    next_token = torch.multinomial(P, num_samples=1)
    if next_token.item() in [50256, 198, 628]:
      break
    gen_ids = torch.cat([gen_ids, next_token])

0.815280556678772  there The the There a
0.9942699670791626  is are's isn�
0.9793866276741028  a an no not cheap
0.463838666677475  place cheap coffee Chinese good
0.8680830001831055  customer rated quality- rating
0.6948366761207581  Chinese place coffee restaurant and
0.8101414442062378  called near with in which
0.8768625855445862  called which the with that
0.976472020149231  The the Burger Bur '
0.995140790939331  Eagle Eagles eagle Ee
0.8781904578208923  which and that with,
0.7374666929244995  which it and a with
0.906307578086853  serves is has sells offers
0.6624318361282349  a Chinese expensive in cheap
0.9308741092681885  the a Burger to city
0.936227560043335  the Burger a city burger
0.9520463347434998  Burger city burger Bur City
0.9991942644119263  King king Kings K

0.7686624526977539  and, which in

0.7663959264755249  it has is which the
0.8171210289001465  a Chinese an good food
0.6286001205444336  that which from of like
0.887141227722168  the China Chinese a all
0.

In [40]:
print('Meaning Representation: ' + cur_mr)
print('Output:' + tokenizer.decode(gen_ids))

Meaning Representation: name[The Eagle], eatType[coffee shop], food[Chinese], priceRange[more than £30], customer rating[high], area[city centre], familyFriendly[yes], near[Burger King]
Output: there is a high rated restaurant nearby called The Eagle, which is near to the Burger King and has food from Chinese and coffee shop having a low customer rating of high and it is expensive.


In [277]:
C[-5000:].sum()

tensor(0.0007, device='cuda:1')

In [332]:
F.abs().min()

tensor(9.9465e-07, device='cuda:1')

In [278]:
U[-5000:].sum()

tensor(0.0041, device='cuda:1')

In [279]:
R[-5000:].sum()

tensor(-0.0034, device='cuda:1')

In [280]:
PR2 = softmax(R, dim=-1)

In [281]:
PR1 = softmax(R1, dim=-1)

In [282]:
PR1[-5000:].sum()

tensor(0.0992, device='cuda:1')

In [283]:
PR2[-5000:].sum()

tensor(0.0992, device='cuda:1')

In [190]:
print('Meaning Representation: ' + cur_mr)
print('Output:' + tokenizer.decode(gen_ids))

Meaning Representation: name[The Eagle], eatType[coffee shop], food[Chinese], priceRange[more than £30], customer rating[high], area[city centre], familyFriendly[yes], near[Burger King]
Output:bury asbestos Monkey germ Marcos services holding vendor violate cannedicideeeks briefingstags exploitRIPT795 legitimate preferential Super Gardens Groups WeeksBS most Ca NAD replay 750 Pepabilitynick sque branded inefficientOr decipherres ranch dup


In [170]:
gen_ids

tensor([  383, 18456,   318,   287,   262,  1748,  7372,   290,   340,   468,
          257,  1029,  6491,  7955,   286,  1029,   290,   340,   318,  5789,
          198,   198,  3666,  1459,  2438,   318,   355,  5679,    25,   198,
            2,  9654,  2393,   198,   259,  8979,   796,  1280, 10786, 27432],
       device='cuda:1')

In [8]:
p=0.5

In [9]:
with torch.no_grad():
  probs = softmax(model(conditional_ids).logits[-1], dim=-1)
  sorted_probs, sorted_indices = torch.sort(probs, descending=True)
  cumulative_probs = sorted_probs.cumsum(dim=-1)
  sorted_indices_to_keep = cumulative_probs < p
  sorted_indices_to_keep[..., 0] = True
  indices_to_keep = sorted_indices_to_keep.scatter(0, sorted_indices, sorted_indices_to_keep)

In [12]:
with torch.no_grad():
  U = softmax(model(unconditional_ids).logits[-1], dim=-1)
  C = softmax(model(conditional_ids).logits[-1], dim=-1)
  cond_p_mask = top_p_mask(C, p)
  U, C = U*cond_p_mask, C*cond_p_mask

In [17]:
tokenizer.decode(torch.topk(U, 1).indices)

' there'

In [16]:
U.gt(0).sum()

tensor(1, device='cuda:1')

In [ ]:
torch.ra

In [ ]:
U = model(unc_inp).logits[-1]
C = model(con_inp).logits[-1]
up = min(C.min(), U.min())
C_pos, U_pos = C+up, U+up
R = C_pos-U_pos
F = U + s*R
P = softmax(F, dim=-1)